In [3]:
from nltk.corpus import stopwords
import nltk
stopwords = set(stopwords.words('russian'))

import pymorphy2
import re
morph = pymorphy2.MorphAnalyzer()

def to_tokens(text):
    no_punct = re.sub('[!,.:;?]', '', text)
    pymorphy_text = list(map(lambda x: morph.parse(x)[0].normal_form, no_punct.split() ))
    text_no_stopwords = [word for word in pymorphy_text if word not in stopwords]
    return text_no_stopwords

def get_grams_intersection(res, vac):
    
    tokens_resume = to_tokens(res)
    tokens_vacancy = to_tokens(vac)
    print(tokens_resume[:100])
    
    bigram_resume = set(nltk.ngrams(tokens_resume, 3))
    bigram_vacancy = set(nltk.ngrams(tokens_vacancy, 3))
    
    intersection = bigram_resume & bigram_vacancy
    return intersection
    


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [20]:
import os
import json
cur_dir = os.getcwd()
file_name = cur_dir + '/case_2_data_for_members.json'
with open(file_name, 'r') as file:
    data = json.load(file)

differences = []

for job in data:
    failed_job_intersections = []
    success_job_intersections = []
    resume = job['vacancy']['description']
    tokens_resume = to_tokens(resume)
    unigram_resume = set(nltk.ngrams(tokens_resume, 3))
    
    for failed in job['failed_resumes']:
        if 'experienceItem' in failed:
            unigram_experience = set()
            for last_job in failed['experienceItem']:
                
                if last_job['description'] is not None:
                    experience = last_job['description']
                    tokens_experience = to_tokens(experience)
                    unigram_experience = unigram_experience | set(nltk.ngrams(tokens_experience, 3))
                    
                    
            failed_job_intersections.append(len(unigram_experience & unigram_resume))

    for success in job['confirmed_resumes']:
        if 'experienceItem' in success:
            unigram_experience = set()
            for last_job in failed['experienceItem']:

                if last_job['description'] is not None:
                    experience = last_job['description']
                    tokens_experience = to_tokens(experience)
                    unigram_experience = unigram_experience | set(nltk.ngrams(tokens_experience, 3))


            success_job_intersections.append(len(unigram_experience & unigram_resume))
    
    avg_suc = round(sum(success_job_intersections) / len(success_job_intersections), 1)
    avg_failed = round(sum(failed_job_intersections) / len(failed_job_intersections), 1)
    differences.append( avg_suc -  avg_failed)
    #print(success_job_intersections)
    print(avg_suc, avg_failed)
                    
                    
print(sum(differences) / len(differences))                    
                   
    


1.0 0.1
0.0 0.0
0.0 0.1
0.0 0.0
0.0 0.0
0.0 0.3
0.0 0.1
0.0 0.0
0.0 0.0
1.0 0.5
0.0 0.9
1.0 0.5
0.0 0.0
0.0 0.1
0.0 0.0
0.0 0.0
5.0 0.9
0.0 0.2
0.0 0.2
0.0 0.3
0.0 0.1
0.0 0.0
2.0 0.2
0.0 0.0
0.0 0.1
0.0 0.2
0.0 0.4
0.0 0.0
0.0 0.0
0.16551724137931034


Нет зависимости между общими парами слов в резюме/акансии


In [62]:
import re

#вроде норм тема, 95% извлекает правильно
def extract_english_words(text):
    english_word_pattern = re.compile(r'\b[a-zA-Z]+\b')
    english_words = english_word_pattern.findall(text)

    return english_words

#возвращает кем работал, key_skills, extra_skills, сколько работал (в месяцах), возраст
def get_workin_years_extra_skills_positions(resume):
    positions = set()
    key_skills = set(resume['key_skills'].replace('.', '').replace(' ', '').split(',')) if 'key_skills' in resume else set()
    extra_skills = set()
    working_months = 0
    years = 2024 - int(resume['birth_date'].split('-')[0])


    about = resume['about'] if 'description' in resume and  resume['about'] is not None  else ''
    extra_skills = set(extract_english_words(about))
    
    if 'experienceItem' in resume and resume['experienceItem'] is not None:
        for last_job in resume['experienceItem']:
            if 'position' in last_job and last_job['position'] is not None:
                positions.add( last_job['position'])
            
            if last_job['starts'] != None:
                start = list(map(int, last_job['starts'].split('-')))
                ends = list(map(int, last_job['ends'].split('-'))) if last_job['ends'] != None else [2024]
        
                if len(start) < 2:
                    start.append(0)

                if len(ends) < 2:
                    ends.append(0)
        
            
                months = (ends[0] - start[0]) * 12
                months += ends[1] - start[1]        
                working_months += months
                
            descript = last_job['description'] if 'description' in last_job and last_job['description'] is not None else ''
            
            #print(descript)
            skills = set(extract_english_words(descript))
            
            extra_skills = skills | extra_skills
    extra_skills -= key_skills
    return positions, key_skills, extra_skills, working_months, years

test = data[0]['confirmed_resumes'][0]

for item in (get_workin_years_extra_skills_positions(test)):
    print(item)
                
        

{'java-разработчик', 'java-developer'}
{'Git', 'SpringBoot', 'JavaSpringFramework', 'ApacheKafka', 'Hibernate', 'REST', 'PostgreSQL', 'Java', 'Camunda'}
{'Maven', 'Boot', 'jtapi', 'POI', 'Security', 'Spring', 'Github', 'Tomcat', 'Bitbucket', 'to', 'code', 'review', 'Kafka', 'JasperReports', 'market', 'Mockito', 'EJB', 'Docker', 'Swagger', 'legacy', 'Liquibase', 'time', 'jUnit', 'Avaya', 'Data', 'WildFly', 'call', 'Postgres', 'CRM', 'API'}
74
37
